In [1]:
using JuMP
using Cbc
using DataFrames
using Gurobi
using CSV

In [10]:
m = Model(solver=GurobiSolver(MIPGap=.1))
#not taken items (randomly generated)
ntaken = [7, 9, 12, 13, 15, 21, 24, 27, 28, 29, 34, 40, 43, 44, 45, 47, 48, 50, 59, 64, 67, 68, 74, 76, 78, 79, 82, 83, 84, 85, 86, 87, 88, 89, 92, 93, 97, 98, 101]
data = CSV.read("supermarket_data.csv")
simple_dist = CSV.read("distmatrix_simple.csv")

v = data[:,2]

simple_dist = simple_dist[:,2:103]
c = simple_dist
    
@variables m begin
    y[1:102], Bin # 1 if item i is taken
end

@variables m begin
    x[1:102,1:102], Bin # 1 if direct path betweeen i and j
end

@variables m begin
    z[1:102]
end

@variables m begin
    t[1:102,1:102]
end

@constraint(m, y[1] == 1)
@constraint(m, y[102] == 1)
@constraint(m, z[1] == 0)
@constraint(m, sum(y[i] for i in 2:101) <= 15)
@constraint(m, sum(x[i,1] for i in 1:102) == 0)
@constraint(m, sum(x[102,j] for j in 1:102) == 0)

for i in ntaken
    @constraint(m, y[i] == 0)
end

for i in 1:101
    @constraint(m, sum(x[i,j] for j in 2:102) == y[i])
end

for j in 2:102
    @constraint(m, sum(x[i,j] for i in 1:101) == y[j])
end

for j in 2:102
    @constraint(m, sum(t[i,j] for i in 1:101) == z[j])
end

for j in 1:101
    @constraint(m, sum(t[j,k] for k in 2:102) == (z[j] + sum(c[j,k]x[j,k] for k in 2:102)))
end

for j in 1:102
    for k in 2:102
        @constraint(m, t[j,k] >= x[j,k])
        @constraint(m, t[j,k] <= 90x[j,k])
    end
end

@objective(m, Max, sum(v[i]*y[i] for i in 1:102))

solve(m)

Academic license - for non-commercial use only
Optimize a model with 21053 rows, 21012 columns and 92962 nonzeros
Variable types: 10506 continuous, 10506 integer (10506 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+01]
  Objective range  [9e-01, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 16.1700000
Presolve removed 13425 rows and 13448 columns
Presolve time: 0.34s
Presolved: 7628 rows, 7564 columns, 33794 nonzeros
Variable types: 3721 continuous, 3843 integer (3843 binary)

Root relaxation: objective 1.188469e+02, 3692 iterations, 0.36 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  118.84691    0   40   16.17000  118.84691   635%     -    0s
H    0     0                      71.9000000  118.84691  65.3%     -    0s
H    0     0                     117.3400000  118.84691  1.28%  

:Optimal

In [12]:
println("Items Taken (In Order):")
cnode = 1
while cnode != 102
    for i in 1:102
        if getvalue(x[cnode,i]) == 1
            cnode = i
            if cnode != 102
                println(data[cnode,1], "(Item #: ",  cnode - 1, ", ", "Price: ", data[cnode,2] , ")")
            end
            break
        end
    end
end

println("")

println("Objective Value: ", getobjectivevalue(m))

println("Total Time: ", getvalue(z[102]))

Items Taken (In Order):
Coffee Beans(Item #: 1, Price: 6.99)
K-Cups(Item #: 2, Price: 10.99)
Granola(Item #: 7, Price: 5.49)
Olive Oil(Item #: 9, Price: 6.99)
Salami(Item #: 89, Price: 5.99)
Chicken Breast(Item #: 90, Price: 6.98)
Shampoo(Item #: 40, Price: 8.99)
Toilet Paper(Item #: 41, Price: 7.99)
Glaceau SmartWater(Item #: 69, Price: 6.99)
Salmon(Item #: 95, Price: 5.99)
Air Freshner(Item #: 55, Price: 6.99)
Swiffer Refills(Item #: 53, Price: 7.99)
Broom(Item #: 51, Price: 13.99)
Redbull (4)(Item #: 61, Price: 7.99)
Gatorade (12)(Item #: 60, Price: 6.99)

Objective Value: 117.33999999999997
Total Time: 90.0
